In [1]:
from geodata.db.client import WorldDataDB
from geodata.db.models.country import Country
from geodata.db.models.state import State
from geodata.db.models.city import City

In [2]:
from geodata.wikidata.sparql import results_from_query
from geodata.wikidata.querys import query_country_id_wikidata

In [3]:
def reset_collections(db: WorldDataDB) -> None:
    db.db.drop_collection('states')
    db.db.drop_collection('cities')
    db.db.drop_collection('countries')
    db.set_unique_keys()

db = WorldDataDB()
#reset_collections(db)
db.db.list_collection_names()

['countries', 'states', 'cities']

In [4]:
db.cities.coll.find_one()

{'_id': ObjectId('65ff06dc64eabd9f7a4b5eae'),
 'created_time': datetime.datetime(2024, 3, 23, 16, 44, 12, 583000),
 'updated_time': datetime.datetime(2024, 3, 23, 16, 44, 12, 583000),
 'country_code': 'AF',
 'country_id_csc': 1,
 'latitude': 36.68333,
 'longitude': 71.53333,
 'postal_codes_wikidata': [],
 'websites_wikidata': [],
 'city_id_csc': 52,
 'state_id_csc': 3901,
 'city_name': 'Ashkāsham',
 'city_name_native': None,
 'city_name_english': None,
 'state_code': 'BDS',
 'city_id_wikidata': 'Q4805192',
 'postal_codes_wikipedia': [],
 'postal_codes_wikipedia_clean': []}

In [10]:
db.cities.coll.find_one({"city_name_native": "Wetzlar"})

{'_id': ObjectId('65ff06eb64eabd9f7a4bf270'),
 'created_time': datetime.datetime(2024, 3, 23, 16, 44, 27, 736000),
 'updated_time': datetime.datetime(2024, 4, 8, 2, 42, 12, 326000),
 'country_code': 'DE',
 'country_id_csc': 82,
 'latitude': 50.56109,
 'longitude': 8.50495,
 'postal_codes_wikidata': ['35521', '35576', '35586'],
 'websites_wikidata': ['https://www.wetzlar.de/'],
 'city_id_csc': 30230,
 'state_id_csc': 3018,
 'city_name': 'Wetzlar',
 'city_name_native': 'Wetzlar',
 'city_name_english': 'Wetzlar',
 'state_code': 'HE',
 'city_id_wikidata': 'Q4178',
 'postal_codes_wikipedia': ['35576-35586'],
 'postal_codes_wikipedia_clean': ['35576',
  '35577',
  '35578',
  '35579',
  '35580',
  '35581',
  '35582',
  '35583',
  '35584',
  '35585',
  '35586']}

In [31]:
from SPARQLWrapper import SPARQLWrapper, JSON
from geodata.wikidata.sparql import random_user_agent

def obtener_id_municipios(id_ciudad_wikidata):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)

    # Consulta SPARQL para obtener los municipios de la ciudad
    consulta = """
    SELECT ?municipio ?municipioLabel WHERE {
      ?municipio wdt:P31/wdt:P279* wd:Q15284;  # Municipio
                 wdt:P131 wd:""" + id_ciudad_wikidata + """.  # Ciudad
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """
    
    sparql.setQuery(consulta)
    sparql.setReturnFormat(JSON)
    sparql.addCustomHttpHeader("User-Agent", random_user_agent())
    resultados = sparql.query().convert()

    id_municipios = []
    for resultado in resultados["results"]["bindings"]:
        id_municipio = resultado["municipio"]["value"].split("/")[-1]
        id_municipios.append(id_municipio)

    return id_municipios

def obtener_coordenadas_municipio(id_municipio_wikidata):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)

    # Consulta SPARQL para obtener las coordenadas del municipio
    consulta = """
    SELECT ?coordenadas WHERE {
      wd:""" + id_municipio_wikidata + """ wdt:P625 ?coordenadas.  # Coordenadas geográficas
    }
    """
    
    sparql.setQuery(consulta)
    sparql.setReturnFormat(JSON)
    resultados = sparql.query().convert()

    coordenadas = []
    for resultado in resultados["results"]["bindings"]:
        coordenadas_str = resultado.get("coordenadas", {}).get("value", "")
        # Las coordenadas se presentan en formato "Point(latitude, longitude)"
        # Parseamos las coordenadas para obtener la latitud y longitud por separado
        if coordenadas_str:
            latitud, longitud = map(float, coordenadas_str.split("(")[1].split(")")[0].split())
            coordenadas.append({"latitud": latitud, "longitud": longitud})

    return coordenadas

#for model in db.cities.iter_models():
#    if model.id_wikidata is not None:
#        id_municipios = obtener_id_municipios(model.id_wikidata)
#        for id_municipio in id_municipios:
#            coords = obtener_coordenadas_municipio(id_municipio)
#            print("Coords del municipio:", coords)
#            print(model.name)
#            print(model.id_wikidata)
#            print("-"*40)
#    print(model.name)

[obtener_coordenadas_municipio(id_municipio) for id_municipio in obtener_id_municipios("Q4178")]

[]

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_municipality_data(city_id):
    # Configurar el punto de acceso SPARQL de DBpedia
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")

    # Definir la consulta SPARQL
    query = """
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
    PREFIX dbpprop: <http://dbpedia.org/property/>

    SELECT ?municipality ?street ?postalCode
    WHERE {
      ?municipality a dbpedia-owl:CityHall;
                    dbpprop:city ?city;
                    dbpprop:postalCode ?postalCode;
                    dbpprop:address ?street.
      ?city dbpprop:wikidata ?city_id.
      FILTER (?city_id = '%s')
    }
    """ % city_id

    # Asignar la consulta SPARQL
    sparql.setQuery(query)

    # Establecer el formato de salida como JSON
    sparql.setReturnFormat(JSON)

    # Ejecutar la consulta y obtener los resultados
    results = sparql.query().convert()

    # Procesar los resultados
    if results and 'results' in results and 'bindings' in results['results']:
        for result in results['results']['bindings']:
            municipality = result['municipality']['value']
            street = result['street']['value']
            postal_code = result['postalCode']['value']
            print("Municipality:", municipality)
            print("Street:", street)
            print("Postal Code:", postal_code)
    else:
        print("No se encontraron resultados.")

city_id = "Q4178"

# Llamar a la función para obtener los datos de la municipalidad
get_municipality_data(city_id)


In [21]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_municipality_id(city_id):
    # Set up the SPARQL endpoint
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # Define the SPARQL query
    query = """
    SELECT ?municipality ?municipalityLabel WHERE {
      wd:""" + city_id + """ wdt:P131 ?municipality.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    # Set the query and response format
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Execute the query and process the results
    results = sparql.query().convert()
    
    # Extract municipality ID and label from the results
    if 'results' in results and 'bindings' in results['results']:
        bindings = results['results']['bindings']
        if len(bindings) > 0:
            municipality_id = bindings[0]['municipality']['value'].split('/')[-1]
            municipality_label = bindings[0]['municipalityLabel']['value']
            return municipality_id, municipality_label

    return None, None

def get_location_info(municipality_id):
    # Set up the SPARQL endpoint
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # Define the SPARQL query
    query = """
    SELECT ?street ?streetLabel ?address WHERE {
      ?street wdt:P131 wd:""" + municipality_id + """.
      ?street wdt:P6375 ?address.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    # Set the query and response format
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Execute the query and process the results
    results = sparql.query().convert()

    # Extract location information from the results
    location_info = []
    if 'results' in results and 'bindings' in results['results']:
        bindings = results['results']['bindings']
        for binding in bindings:
            street_id = binding['street']['value'].split('/')[-1]
            street_label = binding['streetLabel']['value']
            address = binding['address']['value']
            location_info.append((street_id, street_label, address))

    return location_info


city_id = "Q64"#"Q4178"
municipality_id, municipality_label = get_municipality_id(city_id)
location_info = get_location_info(municipality_id)
if municipality_id:
    print("Municipality ID:", municipality_id)
    print("Municipality Label:", municipality_label)
    print("location_info:", location_info)
else:
    print("Municipality not found.")


Municipality ID: Q183
Municipality Label: Germany
location_info: [('Q15979678', 'University Library of Erlangen-Nürnberg', 'Universitätsstraße 4, 91054 Erlangen'), ('Q870977', "National Council of German Women's Organizations", 'Axel-Springer-Str. 54a, 10117 Berlin'), ('Q41291745', 'Feldkapelle', 'Ca. 1 km nordwestlich der  Wallfahrtskirche, an der Straße zur Hohen Warte'), ('Q55980072', 'Q55980072', 'Martin-Luther-Straße 12, 66111 Saarbrücken'), ('Q101237103', 'Q101237103', 'Ulmenstraße 69, Haus 4, 18057 Rostock'), ('Q125236151', 'U.10 Makerspace', 'Willy-Brandt-Ring 11, 67547 Worms')]


In [ ]:
def query_street_and_number(id_wikidata: str, name: str) -> str | None:
    query = f"""
        SELECT ?direccion WHERE {{
        wd:{id_wikidata} wdt:P6375 ?direccion.
        }}
        """
    results = results_from_query(query=query)
    if len(results["results"]["bindings"]) > 0:
        street = results["results"]["bindings"][0]["street"]["value"]
        number = results["results"]["bindings"][0]["number"]["value"] if "number" in results["results"]["bindings"][0] else None
        return f"{street} {number}", name if number else street, name
    else:
        return None, name

from concurrent.futures import ThreadPoolExecutor, as_completed

with ThreadPoolExecutor(max_workers=5) as pool:
    iter_futures = (pool.submit(query_street_and_number, model.id_wikidata, model.name) \
                    for model in db.cities.iter_models() if model.id_wikidata is not None)
    for future in as_completed(iter_futures):
        r, name = future.result()
        print(f"r={r} | name={name}")

#for model in db.cities.iter_models():
#    if model.id_wikidata is not None:
#        print(f"{model.name} | address={query_municipality_address(model.id_wikidata)}")
#query_street_and_number("Q1486")

### wikipedia by wikidata_id

In [ ]:
from geodata.wikipedia.postal_wikipedia import get_postal_codes_from_wikipedia
'''
for state in db.states.iter_models():
    if state.id_wikidata is not None:
        display(state.model_dump())
        #wikidata_id = "Q4178"
        display(get_postal_codes_from_wikipedia(state.id_wikidata))
        print("-"*40)
'''
get_postal_codes_from_wikipedia("Q692390", verbose=True)

In [ ]:




# Ejemplo de uso
content = """
    This is some text.
    |postal_code = 12345 Some text here
    Another line with |Postal_Code= 54321
    |zipcode=67890
    And another line without postal code
"""

postal_code_lines = extract_postal_code_lines(content)
print("Líneas con códigos postales:", postal_code_lines)


### Search by title wikipedia (dangerous)

In [ ]:
import requests

def get_wikidata_id_from_wikipedia(title):
    base_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "prop": "pageprops",
        "titles": title,
        "formatversion": 2,
        "redirects": True
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if "query" in data and "pages" in data["query"]:
        page = data["query"]["pages"][0]
        if "pageprops" in page and "wikibase_item" in page["pageprops"]:
            return page["pageprops"]["wikibase_item"]
    return None


article_title = "Mecklenburg-Vorpommern"#"Mecklenburg-Vorpommern"
wid = get_wikidata_id_from_wikipedia(article_title)
print("ID de Wikidata:", wid)